### Tipo de grafo selecionado: Grafo não direcionado
### Cálculo do Grau de Centralidade (Degree Centrality)
### Cálculo Centralidade do Autovetor / Valor (Eigenvector)
### Betweenness
### Closeness

In [1]:
import networkx as nx
import ibm_db
import ibm_db_dbi
import pandas as pd
from pyvis.network import Network
from itertools import combinations
import time

### Função cria_grafo

In [2]:
def cria_grafo(user,pwd):
    
    db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID="+user+";PWD="+pwd+";", "", "")
    conn = ibm_db_dbi.Connection(db) 
    
    # Selecionando ocorrência e pessoas de interesse
    query = """ SELECT
                    oc.NRO_INT_OCORRENCIA,
                    poc.NRO_INT_PESSOA_INTERESSE, 
                    tp.TXT_DESCRICAO VINCULO
                FROM
                    GESEG_TARGET.GESEG_VER_OCORRENCIA oc
                    INNER JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO ft ON oc.NRO_INT_FATO = ft.NRO_INT_FATO
                    INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA poc ON oc.NRO_INT_OCORRENCIA = poc.NRO_INT_OCORRENCIA
                    INNER JOIN GESEG_TARGET.GESEG_DIM_TIPO_PARTICIPACAO tp ON poc.NRO_INT_TIPO_PARTICIPACAO = tp.NRO_INT_TIPO_PARTICIPACAO -- Coloquei para descobrir participação pessoa de interesse
                WHERE
                    ft.NRO_INT_FATO IN (201003,201013,201012,201009,201015,201025,201006,202567,202540,202545,202536,202549,202561,202565,202553,202557,201004,201005)
                    AND YEAR(DATA_FATO) IN (2016,2017)
                    AND poc.NRO_INT_TIPO_PARTICIPACAO IN (4,7,8,13,14)
                GROUP BY oc.NRO_INT_OCORRENCIA, poc.NRO_INT_PESSOA_INTERESSE,tp.TXT_DESCRICAO;"""
    
    df = pd.read_sql_query(query, conn)
    # title é utilizado para aparecer o tipo de vínculo quando o mouse passar pela aresta
    df['title'] = df['VINCULO']
    
    #informando quais colunas da tabela serão o source, target e demais atributos;
    D = nx.from_pandas_edgelist(df, source = 'NRO_INT_OCORRENCIA', target = 'NRO_INT_PESSOA_INTERESSE', edge_attr= True, create_using=nx.Graph())
    
    # definindo o atributo 'tipo_no' para todos os nós
    for n in D.nodes:
        D.nodes[n].setdefault('TIPO_NO', 'sem_descricao')
        
    # NÓ OCORRÊNCIA
    #buscando os atributos do vértice 
    query = """SELECT
                    oc.NRO_INT_OCORRENCIA,
                    ft.DESC_FATO Fato,
                    DATA_REGISTRO,
                    HORA_REGISTRO,
                    DATA_FATO,
                    HORA_FATO,
                    count(*) envolvimentos,
                    count(DISTINCT poc.NRO_INT_PESSOA_INTERESSE) pessoas_envolvidas
                FROM
                    GESEG_TARGET.GESEG_VER_OCORRENCIA oc
                    INNER JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO ft ON oc.NRO_INT_FATO = ft.NRO_INT_FATO
                    INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA poc ON oc.NRO_INT_OCORRENCIA = poc.NRO_INT_OCORRENCIA
                WHERE
                    ft.NRO_INT_FATO IN (201003,201013,201012,201009,201015,201025,201006,202567,202540,202545,202536,202549,202561,202565,202553,202557,201004,201005)
                    AND YEAR(DATA_FATO) IN (2016,2017)
                    AND poc.NRO_INT_TIPO_PARTICIPACAO IN (4,7,8,13,14)
                GROUP BY oc.NRO_INT_OCORRENCIA, DATA_REGISTRO, DATA_FATO, ft.DESC_FATO, HORA_FATO, HORA_REGISTRO;"""

    #TRANSFERINDO DADOS PARA TABELA
    df_node_ocor = pd.read_sql_query(query, conn)
    
    #Criando shape e cor diferente para os nós ocorrência
    df_node_ocor['TIPO_NO'] = 'ocorrencia'
    df_node_ocor['NUMERO'] = df_node_ocor['NRO_INT_OCORRENCIA']
    df_node_ocor['shape'] = 'triangle'
    df_node_ocor['color'] = 'orange'
    df_node_ocor['label'] = ' '
    df_node_ocor['title'] = 'Ocorrência: ' + df_node_ocor['NRO_INT_OCORRENCIA'].astype(str) + '\n Fato: ' + df_node_ocor['FATO']

    #formatando o atributo Data
    df_node_ocor['DATA_FATO'] = pd.to_datetime(df_node_ocor['DATA_FATO'])
    df_node_ocor['DATA_FATO'] = df_node_ocor['DATA_FATO'].dt.strftime('%Y-%m-%d')
    df_node_ocor['DATA_REGISTRO'] = pd.to_datetime(df_node_ocor['DATA_REGISTRO'])
    df_node_ocor['DATA_REGISTRO'] = df_node_ocor['DATA_REGISTRO'].dt.strftime('%Y-%m-%d')
    
    #transformando a tabela em um dicionário para atribuir os dados aos vértices 'ocorrencia'
    node_ocorr_atrib = df_node_ocor.set_index("NRO_INT_OCORRENCIA").T.to_dict()
    # atributo 
    nx.set_node_attributes(D, node_ocorr_atrib)
    
    #NÓ PESSOA DE INTERESSE
    # atributos do nó
    query = """SELECT
                    DISTINCT pi.NRO_INT_PESSOA_INTERESSE,
                    pi.NOME
                FROM
                    GESEG_TARGET.GESEG_VER_PESSOA_INTERESSE pi
                WHERE
                    NRO_INT_PESSOA_INTERESSE IN (
                    SELECT 
                        poc.NRO_INT_PESSOA_INTERESSE
                    FROM
                        GESEG_TARGET.GESEG_VER_OCORRENCIA oc
                        INNER JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO ft ON oc.NRO_INT_FATO = ft.NRO_INT_FATO
                        INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA poc ON oc.NRO_INT_OCORRENCIA = poc.NRO_INT_OCORRENCIA
                    WHERE ft.NRO_INT_FATO IN (201003,201013,201012,201009,201015,201025,201006,202567,202540,202545,202536,202549,202561,202565,202553,202557,201004,201005)
                        AND YEAR(DATA_FATO) IN (2016,2017)
                        AND poc.NRO_INT_TIPO_PARTICIPACAO IN (4,7,8,13,14)
                    );"""
    
    #TRANSFERINDO DADOS PARA TABELA
    df_node_pi = pd.read_sql_query(query, conn)
    
    #Retirando o label dos nós
    df_node_pi['TIPO_NO'] = 'pessoa'
    df_node_pi['label'] = ' '
    df_node_pi['title'] = df_node_pi['NOME']
    df_node_pi['NUMERO'] = df_node_pi['NRO_INT_PESSOA_INTERESSE']
    
    #transformando a tabela em um dicionário para atribuir os dados aos vértices 'pesso interesse'
    node_ocorr_atrib2 = df_node_pi.set_index("NRO_INT_PESSOA_INTERESSE").T.to_dict()
    # definindo atributos
    nx.set_node_attributes(D, node_ocorr_atrib2)
    
    ### Co- Autores
    #Criando vínculo co-autores
    for n in D.nodes:
        if D.nodes[n]['TIPO_NO'] == 'ocorrencia':
            lista_vizinhos = D.neighbors(n)
            liga_vizinhos = combinations(lista_vizinhos, 2) 
            D.add_edges_from(liga_vizinhos, VINCULO ='co-autores')
            
    # Verificando quais nós possuem mais de uma ligação 'co-autor' entre eles
    #duplos = [(u, v, k, d) for u, v, k, d in D.edges(data=True, keys = True) if d['VINCULO'] == 'co-autores' and k != 0]

    # formatando os edges que desejamos excluir
    #duplos_format = []
    #for i in range(len(duplos)):
        #duplos_format.append(duplos[i][:3])
        
    # removendo ligações de co-autores duplicados
    #D.remove_edges_from(duplos_format)
    
    # colocando o atributo 'title' para aparecer qual é a relação quando o mouse parar em cima da aresta
    for u,v,d in D.edges(data=True):
        if d['VINCULO'] == 'co-autores':
            D[u][v]['title'] = 'co-autores'
            
    conn.close()
    return D;

In [3]:
# plotar o grafo
def plot_g_pyviz(G, name='out.html', height='300px', width='100%',notebook=True, directed=False):
    g = G.copy() # some attributes added to nodes
    net = Network(notebook=notebook, directed=directed, height=height, width=width)
    opts = '''
        var options = {
          "physics": {
            "forceAtlas2Based": {
              "gravitationalConstant": -100,
              "centralGravity": 0.11,
              "springLength": 100,
              "springConstant": 0.09,
              "avoidOverlap": 1
            },
            "minVelocity": 0.75,
            "solver": "forceAtlas2Based",
            "timestep": 0.22
          }
        }
    '''

    net.set_options(opts)
    # uncomment this to play with layout
    # net.show_buttons(filter_=['physics'])
    net.from_nx(g)
    return net.show(name)

### Criando grafo

In [4]:
# CRIANDO GRAFO

#COLOCAR CREDENCIAIS para acessar o banco
# chamando função
G = cria_grafo(user, pwd)
tempoExec = time.time() - t1
print("Tempo de execução criação de grafo: {} segundos".format(tempoExec))

Tempo de execução criação de grafo: 16.37013530731201 segundos


In [5]:
# verificando se todos os nós estão com o 'tipo' definido
teste = (n for n in G if G.nodes[n]['TIPO_NO']== 'sem_descricao')
print(sorted(teste))

['4025CAD23B7ED014F4A421D5866337C7']


In [6]:
# Verificando se os edges 'co-autores' duplicados foram excluídos
G['1495C1291F438A12CBE7284F2383F2F1']['71B94064FD47CA2D303503957F0205F2']

{'VINCULO': 'co-autores', 'title': 'co-autores'}

In [7]:
# Testando co-autores // ,32309971
T2= nx.generators.ego_graph(G, 31025120 , radius=1, center = True, undirected = True)

# plotar o grafo
plot_g_pyviz(T2,name='multigrafo2016.html' ,notebook = True, directed = False)

In [8]:
print('número de vértices: ',G.number_of_nodes())
print('número de arestas: ',G.number_of_edges())

número de vértices:  18367
número de arestas:  20953


In [ ]:
# Removendo os nós do tipo " ocorrencia"
del_ocorrencias = [n for n in G.nodes if G.nodes[n]['TIPO_NO'] == 'ocorrencia']
G.remove_nodes_from(del_ocorrencias)

In [16]:
# Verificando se os nós foram removidos
T2= nx.generators.ego_graph(G, '1495C1291F438A12CBE7284F2383F2F1' , radius=1, center = True, undirected = True)
# plotar o grafo
plot_g_pyviz(T2,name='grafo2016.html' ,notebook = True, directed = False)

In [17]:
print('número de vértices: ',G.number_of_nodes())
print('número de arestas: ',G.number_of_edges())

número de vértices:  10490
número de arestas:  8197


In [18]:
# Comunidades

import community

partition = community.best_partition(G)
values = [partition.get(node) for node in G.nodes()]
print("Número de comunidades encontradas: ", len(set(values)))

Número de comunidades encontradas:  6348


In [19]:
import numpy as np

deg = nx.degree(G)
deg_seq = [d for n,d in deg]

nos = G.number_of_nodes()
grau_med = sum(deg_seq)/ nos

print("Grau médio dos nós: %.2f" % grau_med)

Grau médio dos nós: 1.56


Abaixo do parâmetro de Molloy-Reed; a rede não possui um componente gigante.

In [21]:
# Densidade: o quão esparsa é a rede; nível de conectividade
# Conforme o artigo: "em uma rede "brilhante" com um grande número de conexões entre criminosos, 
# a investigação ou captura de um ator ajudaria as autoridades a extrair informações críticas sobre a estrutura da rede.
# Uma rede mais escura, no entanto, embora oculte a estrutura das investigações, retardaria a transferência de informações
# dentro da rede devido aos caminhos mais longos entre os criminosos."
dens = nx.density(G)
print("Densidade do grafo: {:04.4f}".format(dens))

Densidade do grafo: 0.0001


In [22]:
# Eficiência da rede: se as conexões estão contribuindo para a redução do tamanho dos caminhos da rede
# Conforme o artigo: "a eficiência da rede quantifica a troca de informações"
efic = nx.global_efficiency(G)
print("Eficiência do grafo: {:04.4f}".format(efic))

Eficiência do grafo: 0.0006


In [ ]:
# Coeficiente de aglomeração: coesão do grupo a que o nó pertence. Quanto mais os vizinhos do nó se conhecerem,
#maior é o coeficiente de aglomeração, mais coeso é o grupo.
md_ca = nx.average_clustering(G)
md_cc = nx.average_shortest_path_length(G)
print("Coeficiente de aglomeração médio: {:04.2f}".format(md_ca))
print("Comprimento médio de caminho mais curto: {:04.2f}".format(md_cc))

Conforme o artigo: "Redes de cauda pesada apresentam muitos indivíduos fracamente relacionados e alguns poucos indivíduos-chave (também chamados de alvos-chaves) que são muito relacionados (também chamados de hubs)."

## Algoritmos de Centralidade

### Grau de centralidade 
É uma medida do número de links diretos para outros nós na rede. É uma medida local, pois seu valor é calculado com base no número de links que um nó tem com os outros nós diretamente adjacentes a ele. Atores em redes sociais com um alto grau de centralidade servem como hubs e como principais canais de informação. Nas redes sociais, por exemplo, a distribuição de graus de nós segue uma distribuição de leis de potência, o que significa que muito poucos nós têm um número extremamente grande de conexões. Naturalmente, esses nós de alto grau têm mais impacto na rede do que outros nós e, portanto, são considerados mais importantes.

In [24]:
# grau de centralidade
t1 = time.time()
dg = nx.degree_centrality(G)
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')


Tempo de execução: 0.010 segundos
 


In [26]:
# Tabela
d = dict(G.nodes(data="title", default=1))
id_nome = pd.DataFrame(list(d.items()),columns=['Num_Pessoa_Interesse', 'title'])
id_nome.set_index('Num_Pessoa_Interesse',inplace=True)

aux_dg = pd.DataFrame(list(dg.items()), columns=['Num_Pessoa_Interesse', 'Grau de Centralidade'])
aux_dg.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_dg = pd.concat([id_nome, aux_dg], axis=1)
rankings_dg.columns = ['Nome','Grau de Centralidade'] 
rankings_dg.sort_values('Grau de Centralidade', ascending = False, inplace = True)

N = 10490
rankings_dg['Posição'] = range(1, N + 1)

rankings_dg.head(10).round(4)

,Nome,Grau de Centralidade,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.0083,1
1495C1291F438A12CBE7284F2383F2F1,MAGNO BERLIM DA SILVA CARDOSO,0.0041,2
A78C1C54C25E3060E8928CDE8116786F,DOUGLAS GONCALVES ROMANO DOS SANTOS,0.0039,3
76E93116575CD38FC73B37F6BC241ECB,JULIANO DA SILVA FRAGA,0.0033,4
04E6920BCE6144A8BCCC663F6E15B1CA,LUIS ROBERTO DA SILVA SILVA,0.0033,5
71B94064FD47CA2D303503957F0205F2,DENISON RICARDO MONTEIRO DOS SANTOS,0.0033,6
6E7C0DA4D145EE7CE950C0B16F2A2DB6,WILLIAM OTAVIO OLIVEIRA DA SILVA,0.0031,7
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.0030,8
977F331FE83A09B0C2C2A04B93029CB5,ADRIANO SCHULTZ DE ASSIS,0.0029,9


### Algoritmo Eigenvector (Centralidade do autovetor)
A medida de Centralidade do autovetor descreve a centralidade de um nó em relação à estrutura global da rede. Ela atribui pontuações relativas a todos os nós na rede com base no conceito de que as conexões para nós com alta pontuação contribuem mais do que as conexões para nós com baixa pontuação.
Ela mede a extensão em que um nó é conectado a nós bem conectados.
Nós com alta pontuação são considerados influentes por estarem conectados a nós importantes. Enquanto permanecem em grande parte nas sombras, eles podem explorar o poder dado por conhecer nós bem conectados.

In [27]:
t1 = time.time()
# centralidade do autovetor
ec = nx.eigenvector_centrality_numpy(G) # função mais rápida; não precisa converter
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')

Tempo de execução: 0.129 segundos
 


In [28]:
# Tabela
aux_ec = pd.DataFrame(list(ec.items()), columns=['Num_Pessoa_Interesse', 'Eigenvector'])
aux_ec.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_ec = pd.concat([id_nome, aux_ec], axis=1)
rankings_ec.columns = ['Nome','Eigenvector'] 
rankings_ec.sort_values('Eigenvector', ascending = False, inplace = True)

N = 10490
rankings_ec['Posição'] = range(1, N + 1)
rankings_ec.head(10).round(4)

,Nome,Eigenvector,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.3320,1
1495C1291F438A12CBE7284F2383F2F1,MAGNO BERLIM DA SILVA CARDOSO,0.2548,2
A78C1C54C25E3060E8928CDE8116786F,DOUGLAS GONCALVES ROMANO DOS SANTOS,0.2401,3
71B94064FD47CA2D303503957F0205F2,DENISON RICARDO MONTEIRO DOS SANTOS,0.2313,4
76E93116575CD38FC73B37F6BC241ECB,JULIANO DA SILVA FRAGA,0.2223,5
6E7C0DA4D145EE7CE950C0B16F2A2DB6,WILLIAM OTAVIO OLIVEIRA DA SILVA,0.2077,6
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.1936,7
C348D60507F5D21F955EE964FCCCEBA0,DOUGLAS LOHAN RODRIGUES DA SILVA,0.1860,8
C43BE20422FB709F7FF721EF132208CB,GABRIEL ANTONIO DE OLIVEIRA BITTENCOURT,0.1752,9


### Algoritmo Betweenness (Centralidade de Interdependência)
Mede o número de caminhos mais curtos em que o nó se encontra. A Centralidade de Interdependência pode ser descrita como quão
importante um nó é, como um link entre diferentes redes. Nós com alta centralidade de interdependência controlam o fluxo de informação porque eles formam pontes críticas entre outros nós ou grupos de nós.
A Centralidade de Interdependência mostra quais nós são prováveis caminhos de informação e pode ser usado para determinar como um grafo irá separar os nós. Da mesma forma, é uma maneira de identificar aqueles que atuam como pontes (também chamadas de chaves de limite) entre dois ou mais segmentos de grafos que, de outra forma, não seriam capazes de se comunicar
entre si. Essa medida toca na importância que um ator pode ganhar estando no meio das comunicações sociais de uma rede e em que medida, em uma sociedade, ele é necessário como elo na cadeia de contatos.

In [29]:
t1 = time.time()
bc = nx.betweenness_centrality(G)
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')


Tempo de execução: 67.523 segundos
 


In [30]:
# Tabela
aux_bc = pd.DataFrame(list(bc.items()), columns=['Num_Pessoa_Interesse', 'Betweenness'])
aux_bc.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_bc = pd.concat([id_nome, aux_bc], axis=1)
rankings_bc.columns = ['Nome','Betweenness'] 
rankings_bc.sort_values('Betweenness', ascending = False, inplace = True)

N = 10490
rankings_bc['Posição'] = range(1, N + 1)
rankings_bc.head(10).round(4)

,Nome,Betweenness,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.0012,1
0765A653C03AC699E2322F5280FE2CBD,WAGNER REIS DA SILVA,0.0010,2
B6C15C650538E504594A9CC68070241B,MAXIMILIANO DA SILVA PACHECO,0.0008,3
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.0008,4
F55E3058C48FB0ACD18812B5B230578F,ROMAR GONCALVES GOMES,0.0006,5
938287B54D1F92232CB490C5890533E3,MARCOS DA SILVA OLIVEIRA,0.0006,6
1D17711922B088BB40A63B849474B6C4,BRUNO FERNANDO SANHUDO TEIXEIRA,0.0005,7
81925BF90874257538F1C626C6F362D3,EDINILSON SILVA DOS SANTOS,0.0004,8
04E6920BCE6144A8BCCC663F6E15B1CA,LUIS ROBERTO DA SILVA SILVA,0.0003,9


In [ ]:
bc_filt = rankings_bc[rankings_bc["Nome"].str.contains('CASSIANO PIRES DA SILVA')==True]
bc_filt

### Algoritmo Closeness (Centralidade de Proximidade)
Esta medida de centralidade tenta expressar a importância de um vértice pelo fato dele estar próximo a muitos outros vértices. Esse vértice pode ser importante se estiver relativamente próximo do conjunto restante dos demais vértices na rede. A centralidade de proximidade é importante para entender a disseminação de informações em redes, de modo que a distância entre um nó em particular e outros tenha um efeito sobre como esse nó pode receber ou enviar informações para outros nós. Nas redes de informação, a proximidade revela quanto tempo leva para que um pouco de informação flua de um nó para outro na rede. Nós de alta pontuação geralmente têm caminhos mais curtos para o restante dos nós na rede.

In [31]:
t1 = time.time()
cc = nx.closeness_centrality(G)
tempoExec = time.time() - t1

print("Tempo de execução: {} segundos".format("%.3f" %tempoExec))
print(' ')

Tempo de execução: 1.299 segundos
 


In [32]:
# Tabela
aux_cc = pd.DataFrame(list(cc.items()), columns=['Num_Pessoa_Interesse', 'Closeness'])
aux_cc.set_index('Num_Pessoa_Interesse',inplace=True)

rankings_cc = pd.concat([id_nome, aux_cc], axis=1)
rankings_cc.columns = ['Nome','Closeness'] 
rankings_cc.sort_values('Closeness', ascending = False, inplace = True)

N = 10490
rankings_cc['Posição'] = range(1, N + 1)
rankings_cc.head(10).round(4)

,Nome,Closeness,Posição
Num_Pessoa_Interesse,,,
FCCE57960B6D4FB3A366E2E2C79B65E8,JOSE DALVANI NUNES RODRIGUES,0.0136,1
04E6920BCE6144A8BCCC663F6E15B1CA,LUIS ROBERTO DA SILVA SILVA,0.0124,2
9B7F3C93D1827BD878FD59888217D131,JEFERSON OLIVEIRA ALMEIDA,0.0124,3
0765A653C03AC699E2322F5280FE2CBD,WAGNER REIS DA SILVA,0.0122,4
1495C1291F438A12CBE7284F2383F2F1,MAGNO BERLIM DA SILVA CARDOSO,0.0121,5
A78C1C54C25E3060E8928CDE8116786F,DOUGLAS GONCALVES ROMANO DOS SANTOS,0.0119,6
76E93116575CD38FC73B37F6BC241ECB,JULIANO DA SILVA FRAGA,0.0119,7
71B94064FD47CA2D303503957F0205F2,DENISON RICARDO MONTEIRO DOS SANTOS,0.0119,8
6E7C0DA4D145EE7CE950C0B16F2A2DB6,WILLIAM OTAVIO OLIVEIRA DA SILVA,0.0118,9


In [ ]:
#V.clear()